In [ ]:
from PIL import Image
from pdf2image import convert_from_path
from PyPDF2 import PdfFileReader, PdfFileWriter
import os
from paddleocr import PaddleOCR
import shutil
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
pdf_path = r'C:\Users\Nirbhayj\Desktop\Honakar\input'
image_path = r'C:\Users\Nirbhayj\Desktop\Honakar\results'
output_path = r'C:\Users\Nirbhayj\Desktop\Honakar\final_result_8'
split_path = r'C:\Users\Nirbhayj\Desktop\Honakar\split_path'
tracker_file = r'C:\Users\Nirbhayj\Desktop\honakar\page_counter.txt'

# To convert the pdf to image


def poppler_img(page_len):
    images = []
    break_ii = 0
    for fil in os.listdir(split_path):
        print(fil)
        image = convert_from_path(os.path.join(
            split_path, fil), poppler_path=r'C:\Users\Nirbhayj\poppler-0.68.0_x86\poppler-0.68.0\bin')
        images.append(image)
        # del (image)
        break_ii += 1
        if break_ii > page_len:
            break
    return images


def save_images(images, st_ii, page_len):
    max_pages=300000
    num_digits = len(str(max_pages))
    for i in range(len(images)):
        page_number=st_ii+i+1
        images[i][0].save(os.path.join(image_path, 'page' + f'{page_number:0{num_digits}}.jpg'), 'JPEG')


input_pdfname = os.listdir(pdf_path)

# path is the pdf file path

# to split the PDF file.


def pdf_splitter(pdffile):
    pdf = PdfFileReader(pdffile)
    len_pages = pdf.getNumPages()
    if os.path.exists(tracker_file):
        with open(tracker_file) as f:
            last_str = f.read().strip() 
            if last_str:
                last_page = int(last_str) 
            else:
                last_page = 0
    
    for page in range(last_page, pdf.getNumPages()):
        pg = page + 1
        max_pages = 100000
        num_digits=len(str(max_pages))
        pdf_writer = PdfFileWriter()
        pdf_writer.addPage(pdf.getPage(page))
        output_filename ='{}\pdf_page_{}.pdf'.format(
            split_path, f'{pg:0{num_digits}}')
        with open(f'{output_filename}', 'wb') as out:
            pdf_writer.write(out)
        print('Created: {}'.format(output_filename))
        images = poppler_img(page)
        save_images(images, 0, 1)
        paddle_ocr(image_path,output_filename)
        with open(tracker_file, 'w') as f:
            f.write(str(pg))
        delete_files(split_path)
        delete_files(image_path)
    open(tracker_file, 'w').truncate()
#     return len_pages


def paddle_ocr(image_path, file1):
    ocr_reader = PaddleOCR(use_angle_cls=True, lang="en", page_num=0)
    for file in os.listdir(image_path):
        result = ocr_reader.ocr(os.path.join(image_path, file), cls=True)
        file_name = os.path.basename(file1)
        txt_name = file_name.replace(".pdf", ".txt")  
        new_name = "page_{}.txt".format(txt_name.split('_')[2]) 
        result_file = open(os.path.join(output_path, new_name), 'w')
        for idx in range(len(result)):
            res = result[idx][1][0]
                # print(res)
            for line in res:
                result_file.write(str(line))
            result_file.write("\n")
#             print(text_output)
#         result_file.close()


def delete_files(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


if __name__ == '__main__':

    pdfFile = os.path.join(pdf_path, input_pdfname[0])
    pdf_splitter(pdfFile)